## ME 481: Whole Body Biomechanics - Balance Lab Pre-Lab Assignment

This notebook will prepare you for Lab 1 in ME 481: Whole Body Biomechanics. Before coming to the lab, you'll learn about the equipment we use, the concepts of balance and postural control, and the signal processing techniques needed to analyze force plate data.

**How to use this notebook:**
- Read each section's short background.
- Run the paired example code cell(s). **Note: You will be asked to complete some of the code yourself!**
- Answer the reflection/critical-thinking prompts before moving on.
- Submit this completed notebook (**as HTML and .ipynb**) to Canvas.

**Course Context:** This lab is the first in a series where you'll develop skills in biomechanical measurement, signal processing, and data analysis using real laboratory equipment. The focus is on understanding human balance and postural control through force plate data.

## Lab Overview and Learning Objectives

### What is This Lab About?

Reduced postural control (balance) is a symptom of many neuromuscular conditions including Parkinson’s disease, multiple sclerosis, and stroke. As such, quantifying balance is common when studying many such conditions. In this lab, we will investigate how postural control (balance) is affected when sensory input is disrupted. To accomplish this, each student will perform a balance test under two conditions – eyes open (EO) and eyes closed (EC). To increase the differences that we observe between the two conditions, we will introduce further instability by completing the tests in tandem stance (heel-to-toe) instead of a standard quiet stance.

### By the end of the lab, you will be able to:

1. **Understand force plate technology** - Translate raw voltage signals from a force plate into corresponding forces and moments
2. **Implement Signal Processing Techniques** - Frequency-domain analysis of timeseries signals and low-pass filtering to eliminate high-frequency noise in data
3. **Calculate balance metrics** - Calculate center-of-pressure (CoP) position and construct stabilogram plots
4. **Evaluate postural control** - compute and report common balance metrics used in scientific literature using your recorded data for two testing conditions


## Data Used in this Pre-Lab Assignment

The data used in this pre-lab assignment was collected using the AMTI force plate that you will use in the lab. The data comes from a tandem-stance (heel-to-toe) balance test performed by an ME481 teaching assistant. The balance test was completed under two conditions - eyes open (EO) and eyes closed (EC). The data was collected at a sampling frequency of 1000 Hz for 15 seconds. In the following sections, you will download the data files and use them to complete the pre-lab assignment.

## I. Background: Understanding Balance and Force Plates

<div style="display: flex; gap: 20px; align-items: flex-start;">

<div style="flex: 1;">

### A. Introduction to Force Plates

A **force plate** is an instrument that measures forces and moments (torques) applied to its surface. In our lab, we use an **AMTI BP400600-2000 force plate** that can measure:

**Forces:**
- **Fx** - Force in the left-right (X) direction
- **Fy** - Force in the forward-backward (Y) direction  
- **Fz** - Force in the vertical (Z) direction

**Moments:**
- **Mx** - Moment (torque) around the X-axis
- **My** - Moment (torque) around the Y-axis
- **Mz** - Moment (torque) around the Z-axis

</div>

<div style="flex: 1; text-align: center;">

<img src="https://uofi.box.com/shared/static/rqkvlr2lt47ungyemnqg1jm4mp9jndsa.png" style="max-height: 400px; width: auto; max-width: 100%;">

*Coordinate system definition for our AMTI force plate.*

</div>

</div>

<div style="display: flex; gap: 20px; align-items: flex-start;">

<div style="flex: 1;">


### B. How Does a Force Plate Work?

The force plate contains **four load cells** positioned near each corner. Load cells are transducers that convert mechanical force into a small, measurable change in electrical resistance using bonded strain gauges on a deformable metal element. When the element experiences load, it strains; the strain gauges elongate or compress, changing their resistance.
Here's the measurement process:

1. **Load cells detect strain** - When you stand on the plate, each load cell experiences a small deformation (strain)
2. **Strain gauges convert to electrical signal** - The deformation changes electrical resistance in strain gauges
3. **Signal amplification** - These small electrical changes are amplified to create measurable voltages
4. **Analog-to-digital conversion** - The analog voltage signals are converted to digital values that a computer can record
5. **Calibration matrix applied** - A mathematical calibration (the sensitivity matrix) converts these voltages into actual forces and moments

**Note:** The raw data you see in the force plate text files are **voltages**, not forces! We must apply a calibration matrix to convert voltages to forces.


</div>

<div style="flex: 1; text-align: center;">

<img src="https://uofi.box.com/shared/static/vgubboi2unxj8nk932eagr3sos7r4vo6.jpg" style="max-height: 400px; width: auto; max-width: 100%;">

*Our AMTI force plate has a series of load cells containing strain gauges that allow it to measure forces and moments along all three axes.*

</div>

</div>

### REFLECTION QUESTION SET 1: Balance, Postural Control, and Force Plates (Answer below)

Now that you've learned about balance, the center of pressure, and how force plates work, let's reflect on these concepts:

1. **Connecting balance to force plate measurements:** In your own words, explain why tracking the **center of pressure (CoP)** over time is a useful way to study how well someone maintains balance. What does it tell us about postural stability?

2. **Eyes Open vs. Eyes Closed:** Based on the physiological roles of vision, the vestibular system, and proprioception in balance control, predict: Will the center of pressure move **more** or **less** when someone's eyes are closed compared to when they're open? Why?

3. **Center of Pressure:** A force plate measures forces (Fx, Fy, Fz) and moments (Mx, My, Mz). Which two force/moment components are used to calculate the x-component of CoP? Which two force/moment components are used to calculate the y-component of CoP?


### ANSWERS FOR SET 1:

1.

2.

3.

## II. Setup and Loading Data

### B. Using your Google Drive in Colab

Start by mounting your Google Drive to this temporary runtime.
You will be prompted to sign in to Google Drive when this runs.

In [ ]:
from google.colab import drive
from google.colab import files
import os

# Mount your Google Drive to this temporary session
drive.mount('/content/gdrive')

# Change current working directory to your lab folder
os.chdir('/content/gdrive/MyDrive/ME481-BalanceLab')
print(f"✓ Working directory set to: {os.getcwd()}")

Using the filetree on the left side, you should now be able to see your Google Drive folders under "/content/gdrive/MyDrive/ME481-BalanceLab". If you click on the link, it should open your Google Drive in the filetree on the left automatically.

### C. Import Required Libraries

Run the cell below to import all necessary libraries and set default plotting parameters.

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import welch, butter, filtfilt
import matplotlib.pyplot as plt
import requests
import ipywidgets as widgets
from IPython.display import display
import nbformat
from nbconvert import HTMLExporter

# Render matplotlib outputs inline
%matplotlib inline

# Set default plot parameters
plt.rcParams['ytick.labelsize'] = 'large'
plt.style.use('default')
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 'large'
plt.rcParams['font.family'] = 'Sans'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.autolayout'] = True
plt.rcParams['axes.grid'] = True

print("✓ All libraries imported successfully!")

### D. Download Example Data

This notebook uses example force plate data from a real balance test. These files are raw voltage outputs showing a tandem stance (heel-to-toe) balance test in two conditions: **Eyes Open (EO)** and **Eyes Closed (EC)**.

We'll download example force plate data from U of I Box. The cell below defines a function to download files and then downloads the example balance data.


In [ ]:
def download_file(url, save_path):
    """
    Downloads a file from a given URL and saves it to a specified path.

    Args:
        url (str): The URL of the file to download.
        save_path (str): The local path where the file will be saved.

    Returns:
        str: The path where the file was saved.

    Raises:
        requests.exceptions.RequestException: If an error occurs during the download.
    """
    r = requests.get(url, stream=True)
    r.raise_for_status()
    with open(save_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return save_path


# Download URLs
eyes_closed_url = 'https://uofi.box.com/shared/static/3hogy24nq7nav1xlcg4icjz78iudqohw.txt'
eyes_open_url = 'https://uofi.box.com/shared/static/va809vdz6pznot71htymsr3d77w03fiu.txt'

# Save paths (local)
eyes_open_path = 'tandem_eyes_open.txt'
eyes_closed_path = 'tandem_eyes_closed.txt'

# Download files
download_file(eyes_open_url, eyes_open_path)
download_file(eyes_closed_url, eyes_closed_path)

if os.path.exists(eyes_open_path) and os.path.exists(eyes_closed_path):
  print("✓ Example data files downloaded successfully!")

### E. Load Example Data

Now let's load the example data files we downloaded into Pandas dataframes. We will set the names of the columns manually. Then, we'll show the first few rows of each dataframe and compute some basic information about the data. Run the cell below to load the data.

In [ ]:
# Define column names
cols = ['Time', 'VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz']

print("DATA COLUMNS:")
print("Time  - Time in seconds")
print("VFx   - Voltage representing force in X direction (left-right)")
print("VFy   - Voltage representing force in Y direction (forward-back)")
print("VFz   - Voltage representing force in Z direction (vertical)")
print("VMx, VMy, VMz - Voltage signals for moments around each axis\n")

# Load data
df_EO = pd.read_csv(eyes_open_path, delimiter='\t', skiprows=6, names=cols)
df_EC = pd.read_csv(eyes_closed_path, delimiter='\t', skiprows=6, names=cols)

# Let's print out the first ten rows
print("FIRST TEN ROWS OF EYES OPEN DATAFRAME:")
print(df_EO.head(10))

# Now let's calculate the shape of the datframe
print(f"\nDataframe Shape: {df_EO.shape} (rows, columns)")

# Total time duration of data (last timestamp)
print(f"Time Duration: {df_EO['Time'].iloc[-1]:.2f} seconds")

# Sampling rate of data (1/period)
sampling_rate = 1/(df_EO['Time'].iloc[1] - df_EO['Time'].iloc[0])
print(f"Sampling rate: {sampling_rate:.0f} Hz")

### F. Plot Raw Voltage Data

Now that we have loaded the data, let's visualize the raw voltage signals from the force plate. This will help us understand the nature of the signals before we apply any processing techniques. Run the cell below to plot the raw voltage data for both the Eyes Open and Eyes Closed conditions.



In [ ]:
# Plotting function
def plot_voltage_data(df_EO, df_EC, time_range=(0, 15), figsize=(10, 8), dpi=300, pad=2.0):
    """
    Plot voltage data from two dataframes (EO and EC) over a specified time range.

    This function generates six subplots corresponding to force (Fx, Fy, Fz) and moment (Mx, My, Mz) voltage signals, comparing data from two different conditions (EO and EC).

    Args:
        df_EO (DataFrame): Data containing EO (Eyes Open) condition, with columns 'Time', 'VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz'.
        df_EC (DataFrame): Data containing EC (Eyes Closed) condition, with the same columns as `df_EO`.
        time_range (tuple, optional): The time range (start, end) in seconds for plotting. Defaults to (0, 15).
        figsize (tuple, optional): The figure size as (width, height). Defaults to (10, 8).
        dpi (int, optional): Dots per inch (DPI) setting for the figure resolution. Defaults to 300.
        pad (float, optional): Padding for `tight_layout` to adjust subplot spacing. Defaults to 2.0.

    Raises:
        ValueError: If the input dataframes do not contain the required voltage columns.

    Example:
        >>> plot_voltage_data(df_EO, df_EC, time_range=(10, 25))
    """
    # Create subplots
    fig, axs = plt.subplots(
        6, 1, sharex=True, sharey=False, dpi=dpi, figsize=figsize)

    # Plot data for each subplot
    for i, (col, title) in enumerate(zip(['VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz'],
                                         ['Fx voltage', 'Fy voltage', 'Fz voltage', 'Mx voltage', 'My voltage', 'Mz voltage'])):
        axs[i].plot(df_EO['Time'], df_EO[col],
                    color='tab:blue', alpha=1.0, label='EO')
        axs[i].plot(df_EC['Time'], df_EC[col],
                    color='tab:orange', alpha=1.0, label='EC')
        axs[i].legend(bbox_to_anchor=(1.1, 1), loc='upper right')
        axs[i].set_title(title)

    # Set x-axis label and limits
    plt.xlabel('Time [s]')
    plt.xlim(time_range)

    # Set y-axis label for the middle subplot
    axs[2].set_ylabel('Voltage [V]')
    fig.align_ylabels()

    # Adjust layout
    plt.tight_layout(pad=pad)

    # Show the plot
    plt.show()


plot_voltage_data(df_EO, df_EC, time_range=(0, 15),
                  figsize=(6, 6), dpi=100, pad=0.5)

#### REFLECTION QUESTION SET 2: Observations from Raw Voltage Data (Answer below)

Look at the plot you just generated comparing Eyes Open and Eyes Closed voltage signals across all six channels (Fx, Fy, Fz, Mx, My, Mz):

1. What are the smallest and largest ranges of voltage values you observe across the six channels? Are there any channels that show significantly larger or smaller voltage ranges?

2. Which signals would correspond to shifting your weight anterior/posterior and medial/lateral? Do you see any differences in these signals between EO and EC?

### ANSWERS FOR SET 2

1.

2.

## III: Converting Force Plate Voltages to Forces and Moments
<br>

**Instructions:** To convert measured voltages into forces and moments, you need the inverted sensitiviy matrix, $B$, the gain value from the amplifier, $G$, and the channel input voltage, $V_0$. For this specific force plate and amplifier:

<br>

$$
B = \begin{bmatrix}
2.9007     & 0.0200     & -0.0009     & -0.0253   & -0.0085      & 0.0090    \\
-0.0067     & 2.9024   & -0.0520     & -0.0366   & -0.0149     & -0.0341   \\
0.0046   & -0.0229     & 11.4206   & -0.0055     & 0.0055    & 0.0026      \\
-0.0019     & 0.0035     & -0.0067     & 1.4559    & -0.0053     & -0.0028    \\
0.0036     & 0.0011     & -0.0067     & 0.0018    & 1.1475     & -0.0008    \\
0.0037     & 0.0145   & -0.0032   & 0.0006   & 0.0076     & 0.6188
\end{bmatrix}
$$



```
# This is a python snippet to create the matrix
B = np.array([
    [2.9007,  0.0200, -0.0009, -0.0253, -0.0085,  0.0090],
    [-0.0067, 2.9024, -0.0520, -0.0366, -0.0149, -0.0341],
    [0.0046, -0.0229, 11.4206, -0.0055,  0.0055,  0.0026],
    [-0.0019,  0.0035, -0.0067,  1.4559, -0.0053, -0.0028],
    [0.0036,  0.0011, -0.0067,  0.0018,  1.1475, -0.0008],
    [0.0037,  0.0145, -0.0032,  0.0006,  0.0076,  0.6188]
])
```



$$
G = 4000
$$

$$
V_0 = 10 \text{ volts}
$$

<br>

Assuming at a certain time-point $t_1$ we have a 1x6 voltage row vector $\vec{V_1} = (V_{Fx}, V_{Fy}, \ldots, V_{Mz})$, we can calculate the corresponding forces and moments, $\vec{Y_1} =(F_{x}, F_{y}, \ldots, M_{z})$ using

$$
\vec{Y_1} = \frac{10^6}{GV_0}\vec{V_1}\boldsymbol{B^T}
$$
<br>



### A. Apply the calibration matrix to convert voltages to forces/moments.

Convert the measured voltages into forces and moments and create a 6x1 subplot showing forces and moments for your eyes-open (EO) and eyes-closed (EC) data. Each subplot should have a single force/moment on the y-axis and time on the x-axis. Include a legend to differentiate the EO and EC conditions. **Fill in the missing code in the cell below to complete this task.**

In [ ]:
# Inverted sensitivity matrix, B
B = np.array([
    [2.9007,  0.0200, -0.0009, -0.0253, -0.0085,  0.0090],
    [-0.0067, 2.9024, -0.0520, -0.0366, -0.0149, -0.0341],
    [0.0046, -0.0229, 11.4206, -0.0055,  0.0055,  0.0026],
    [-0.0019,  0.0035, -0.0067,  1.4559, -0.0053, -0.0028],
    [0.0036,  0.0011, -0.0067,  0.0018,  1.1475, -0.0008],
    [0.0037,  0.0145, -0.0032,  0.0006,  0.0076,  0.6188]
])

G = 4000  # Gain
V_0 = 10  # Input voltage
CF = 1e6/(G*V_0)  # Conversion factor

# Get voltages as array-like
V_EO = df_EO[['VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz']].to_numpy()
V_EC = df_EC[['VFx', 'VFy', 'VFz', 'VMx', 'VMy', 'VMz']].to_numpy()
# Compute forces and moments in a vectorized manner
Y_EO = #TODO: YOUR CODE GOES HERE
Y_EC = #TODO: YOUR CODE GOES HERE
# Assign computed values back to DataFrame
df_EO[['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']] = Y_EO
df_EC[['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']] = Y_EC

### B. Plot the calculated forces and moments.

In [ ]:
# Plotting function
def plot_data(df_EO, df_EC, df_cols, plot_titles, y_labels, time_range=(15, 30), figsize=(10, 8), dpi=200, pad=2.0):
    """
    Plot signals from two dataframes (EO and EC) over a specified time range.

    This function generates six subplots corresponding to force (Fx, Fy, Fz) and moment (Mx, My, Mz) signals,
    comparing data from two different conditions (EO and EC).

    Args:
        df_EO (DataFrame): Data containing EO (Eyes Open) condition.
        df_EC (DataFrame): Data containing EC (Eyes Closed) condition, with the same columns as df_EO.
        df_cols (list): List of column names to plot from the dataframes.
        plot_titles (list): List of titles for each subplot.
        y_labels (list): List of y-axis labels for each subplot.
        time_range (tuple, optional): Time range (start, end) in seconds for plotting. Defaults to (15, 30).
        figsize (tuple, optional): Figure size as (width, height). Defaults to (10, 8).
        dpi (int, optional): Dots per inch (DPI) setting for the figure resolution. Defaults to 200.
        pad (float, optional): Padding for tight_layout to adjust subplot spacing. Defaults to 2.0.

    Raises:
        ValueError: If the input dataframes do not contain the required columns.

    Example:
        >>> plot_data(df_EO, df_EC, ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz'], titles, labels, time_range=(10, 25))
    """
    # Create subplots
    fig, axs = plt.subplots(
        6, 1, sharex=True, sharey=False, dpi=dpi, figsize=figsize)

    # Plot data for each subplot
    for i, (col, title, y_label) in enumerate(zip(df_cols,
                                                  plot_titles, y_labels)):
        axs[i].plot(df_EO['Time'], df_EO[col],
                    color='tab:blue', alpha=1.0, label='EO')
        axs[i].plot(df_EC['Time'], df_EC[col],
                    color='tab:orange', alpha=1.0, label='EC')
        axs[i].legend(bbox_to_anchor=(1.1, 1), loc='upper right')
        axs[i].set_ylabel(y_label)
        axs[i].set_title(title)

    # Set x-axis label and limits
    plt.xlabel('Time [s]')
    plt.xlim(time_range)

    # Adjust layout
    plt.tight_layout(pad=pad)

    # Show the plot
    plt.show()


Y_cols = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']
plot_titles = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']
y_labels = ['$F_x$ [N]', '$F_y$ [N]', '$F_z$ [N]',
            '$M_x$ [N-m]', '$M_y$ [N-m]', '$M_z$ [N-m]']

# Plot the first 5 seconds of data
plot_data(df_EO=df_EO, df_EC=df_EC, df_cols=Y_cols, plot_titles=plot_titles,
          y_labels=y_labels, time_range=(0, 5), dpi=100, figsize=(6, 6))

### REFLECTION QUESTION SET 3: Force Plate Calibration (Answer below)
1. Which channel could you get information about the subject's weight from? What is the approximate value of this force in kilograms?
2. Which channel looks the most similar between EO and EC? Which channel looks the most different?
3. Does the EC condition for the $M_y$ show larger variability than EO? Why would that be expected physiologically? Note: remember that this data is from a tandem stance (heel-to-toe) balance test.

### ANSWERS FOR SET 3:

1.

2.

3.

## III. Signal Processing in Biomechanics

### B. Demonstrating Filter Effects

Here is an example of how to design and apply a Butterworth low-pass filter using SciPy in Python:

```python
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y
```

Now let's see how a Butterworth filter affects signal components at different frequencies. Run the cell below to generate a series of plots showing the frequency response of a Butterworth filter and its effect on signals with different frequency components.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, freqz

# Create a Butterworth filter and visualize its frequency response


def butterworth_frequency_response(cutoff_freq, sampling_freq, order=4):
    """
    Compute and return the frequency response of a Butterworth filter.

    Args:
        cutoff_freq (float): Cutoff frequency in Hz
        sampling_freq (float): Sampling frequency in Hz
        order (int): Filter order (default 4 for balance studies)

    Returns:
        frequencies (array): Frequencies in Hz
        magnitude_dB (array): Filter magnitude response in dB
        magnitude_linear (array): Filter magnitude response as linear ratio
    """
    # Design the Butterworth filter
    b, a = butter(N=order, Wn=cutoff_freq, btype='low',
                  fs=sampling_freq, output='ba')

    # Get frequency response
    w, h = freqz(b, a, fs=sampling_freq, worN=10000)

    # Convert magnitude to dB (20 * log10 of magnitude)
    # Add small value to avoid log(0)
    magnitude_dB = 20 * np.log10(np.abs(h) + 1e-10)
    magnitude_linear = np.abs(h)

    return w, magnitude_dB, magnitude_linear


# Create figure with multiple subplots
fig, axs = plt.subplots(2, 2, figsize=(14, 10), dpi=100)

# ============ PLOT 1: Magnitude response (dB) for different filter orders ============
ax = axs[0, 0]
sampling_freq = 1000
cutoff_freq = 10

for order in [1, 2, 4, 6]:
    w, mag_dB, _ = butterworth_frequency_response(
        cutoff_freq, sampling_freq, order=order)
    ax.plot(w, mag_dB, linewidth=2, label=f'Order {order}')

ax.axhline(y=-3, color='red', linestyle='--',
           linewidth=1.5, label='-3 dB point')
ax.axvline(x=cutoff_freq, color='green', linestyle='--',
           linewidth=1.5, label=f'Cutoff ({cutoff_freq} Hz)')
ax.set_xlim(0, 100)
ax.set_ylim(-80, 5)
ax.set_xlabel('Frequency (Hz)', fontsize=11)
ax.set_ylabel('Magnitude (dB)', fontsize=11)
ax.set_title('Butterworth Filter: Effect of Filter Order\n(Cutoff = 10 Hz)',
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

# ============ PLOT 2: Magnitude response (linear) showing attenuation percentage ============
ax = axs[0, 1]
w, _, mag_linear = butterworth_frequency_response(
    cutoff_freq, sampling_freq, order=4)
attenuation_percent = (1 - mag_linear) * 100

ax.plot(w, mag_linear, linewidth=2.5, color='tab:blue',
        label='Signal amplitude (fraction)')
ax.axhline(y=0.707, color='red', linestyle='--', linewidth=1.5,
           label='0.707 (-3 dB, 50% power loss)')
ax.axvline(x=cutoff_freq, color='green', linestyle='--',
           linewidth=1.5, label=f'Cutoff ({cutoff_freq} Hz)')

# Fill the passband and stopband
ax.fill_between(w[w <= cutoff_freq], 0, 1, alpha=0.1,
                color='green', label='Passband')
ax.fill_between(w[w > cutoff_freq], 0, 1, alpha=0.1,
                color='red', label='Stopband')

ax.set_xlim(0, 100)
ax.set_ylim(0, 1.05)
ax.set_xlabel('Frequency (Hz)', fontsize=11)
ax.set_ylabel('Magnitude (Linear Ratio)', fontsize=11)
ax.set_title('Butterworth Filter: Linear Magnitude Response\n(Order 4, Cutoff = 10 Hz)',
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

# ============ PLOT 3: Attenuation (%) across frequency range ============
ax = axs[1, 0]
w, _, mag_linear = butterworth_frequency_response(
    cutoff_freq, sampling_freq, order=4)
attenuation_percent = (1 - mag_linear) * 100

ax.plot(w, attenuation_percent, linewidth=2.5, color='tab:purple')
ax.fill_between(w, attenuation_percent, alpha=0.3, color='tab:purple')
ax.axhline(y=50, color='red', linestyle='--', linewidth=1.5,
           label='50% attenuation (at cutoff)')
ax.axvline(x=cutoff_freq, color='green', linestyle='--',
           linewidth=1.5, label=f'Cutoff ({cutoff_freq} Hz)')

# Add annotations for key frequencies in biomechanics
ax.axvspan(0.5, 3, alpha=0.1, color='blue', label='Postural sway (0.5-5 Hz)')
ax.axvspan(8, 12, alpha=0.1, color='orange',
           label='Physiological tremor (8-12 Hz)')
ax.axvspan(50, 100, alpha=0.1, color='red', label='Sensor noise (>50 Hz)')

ax.set_xlim(0, 100)
ax.set_ylim(0, 105)
ax.set_xlabel('Frequency (Hz)', fontsize=11)
ax.set_ylabel('Attenuation (%)', fontsize=11)
ax.set_title('Butterworth Filter: Attenuation Percentage\n(Order 4, Cutoff = 10 Hz)',
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=9, loc='right')

# ============ PLOT 4: Attenuation at specific frequencies (table-like) ============
ax = axs[1, 1]
ax.axis('off')

# Calculate attenuation at key frequencies
frequencies_of_interest = [1, 2, 3, 5, 10, 15, 20, 50, 100]
w, _, mag_linear = butterworth_frequency_response(
    cutoff_freq, sampling_freq, order=4)

attenuation_at_freqs = []
for freq in frequencies_of_interest:
    idx = np.argmin(np.abs(w - freq))
    atten = (1 - mag_linear[idx]) * 100
    atten_dB = 20 * np.log10(mag_linear[idx])
    attenuation_at_freqs.append(
        [freq, f'{mag_linear[idx]:.3f}', f'{atten:.1f}%', f'{atten_dB:.1f} dB'])

# Create table
table_data = [['Frequency (Hz)', 'Magnitude', 'Attenuation',
               'Attenuation (dB)']] + attenuation_at_freqs
table = ax.table(cellText=table_data, cellLoc='center', loc='center',
                 colWidths=[0.2, 0.25, 0.25, 0.3])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# Style header row
for i in range(4):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Style rows with colors based on frequency region
for i, freq in enumerate(frequencies_of_interest, start=1):
    if freq <= 5:
        color = '#E7F0FF'  # Light blue for sway
    elif freq <= 12:
        color = '#FFF4E7'  # Light orange for tremor
    else:
        color = '#FFE7E7'  # Light red for noise

    for j in range(4):
        table[(i, j)].set_facecolor(color)

ax.text(0.5, 1.15, f'Butterworth Filter (Order 4, Cutoff = {cutoff_freq} Hz, Sampling Rate = {sampling_freq} Hz)',
        ha='center', fontsize=11, fontweight='bold', transform=ax.transAxes)

plt.tight_layout()
plt.show()

#### Understanding the Butterworth Filter Frequency Response Plots

The figure above shows four different ways to visualize how a Butterworth filter behaves across different frequencies. Each subplot provides unique insights into filter performance:

**Plot 1: Effect of Filter Order (Top Left)**
This shows how **filter order affects the sharpness** of the frequency cutoff. All filters shown here have a 10 Hz cutoff, but higher-order filters provide steeper transitions between the passband and stopband. **Order 4 is a common choice** in biomechanics.

**Plot 2: Linear Magnitude Response (Top Right)**
This shows **what fraction of the signal passes through** at each frequency:
- **Passband (green)**: Frequencies below 10 Hz pass through nearly unchanged (magnitude ≈ 1.0)
- **Cutoff point (10 Hz)**: Signal is attenuated to 0.707 of its original amplitude (the -3 dB point, representing 50% power loss)
- **Stopband (red)**: Higher frequencies are progressively attenuated

This visualization helps you understand **how much of a signal at a given frequency is preserved** when using a filter with a cutoff frequency of 10 Hz.

**Plot 3: Attenuation Percentage with Biomechanical Context (Bottom Left)**
This is the inverse of Plot 2, showing **how much signal is removed** (rather than retained) and mapping filter behavior to **physiological phenomena**:
- **Postural sway region (0.5-3 Hz, blue)**: <5% attenuation - signal preserved
- **Physiological tremor (8-12 Hz, orange)**: 30-70% attenuation - partially removed
- **Sensor noise (>50 Hz, red)**: >95% attenuation - effectively eliminated

Note that these frequency bands are approximate and idealized; actual data may vary.

You can see that a 10 Hz cutoff preserves postural sway while removing most noise. If you're studying tremor, you'd need a higher cutoff (15-20 Hz).

**Plot 4: Attenuation Table (Bottom Right)**
This provides **exact numerical values** showing magnitude, attenuation percentage, and attenuation in dB at specific frequencies given a filter of the stated design. For example: "At 5 Hz (upper range of postural sway), the filter attenuates only 1.6%, preserving 98.4% of signal amplitude, while at 50 Hz (sensor noise range), 99.8% attenuation removes nearly all noise."

**Connection to Your Balance Lab Analysis**
For this balance lab, your filtering goals are:
1. **Preserve low-frequency postural sway** - where real CoP movement occurs
2. **Remove high-frequency noise** - from sensors and electrical interference
3. **Choose an appropriate cutoff** to balance these needs

#### Effect of Different Cutoff Frequencies

Let's create a simple example showing how different cutoff frequencies affect a signal:

In [ ]:
# Generate example: sine wave + noise
np.random.seed(42)
sampling_rate = 1000  # Hz
duration = 1.0  # seconds
signal_frequency = 5  # Hz (the "real" signal)

time = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
clean_signal = np.sin(2 * np.pi * signal_frequency * time)
noise = 0.3 * np.random.normal(0, 1, len(time))
noisy_signal = clean_signal + noise


def lowpass_filter(data, cutoff_freq, sampling_rate, order=4):
    """
    Apply a zero-phase Butterworth low-pass filter to a 1D signal.

    Args:
        data (array-like): Input signal to be filtered.
        cutoff_freq (float): Cutoff frequency of the filter in Hz.
        sampling_rate (float): Sampling rate of the signal in Hz.
        order (int, optional): Order of the Butterworth filter. Default is 4.

    Returns:
        ndarray: The filtered signal, same shape as input.

    Example:
        filtered = lowpass_filter(signal, 10, 1000)
    """
    b, a = butter(N=order, Wn=cutoff_freq, btype="low",
                  fs=sampling_rate, output="ba")
    return filtfilt(b, a, data)


# Try different cutoff frequencies
cutoff_frequencies = [2, 5, 10, 50, 100]

fig, axs = plt.subplots(len(cutoff_frequencies), 1,
                        figsize=(8, 6), sharex=True, dpi=100)
fig.suptitle('Effect of Cutoff Frequency on Signal Filtering\n(Real signal: 5 Hz)',
             fontsize=14, fontweight='bold')

for i, fc in enumerate(cutoff_frequencies):
    filtered_signal = lowpass_filter(noisy_signal, fc, sampling_rate)

    axs[i].plot(time, clean_signal, 'g-', alpha=0.6,
                linewidth=2, label='Clean Signal (5 Hz)')
    axs[i].plot(time, noisy_signal, 'k-', alpha=0.4,
                linewidth=1, label='Noisy Signal')
    axs[i].plot(time, filtered_signal, 'r-', linewidth=2,
                label=f'Filtered ({fc} Hz cutoff)')

    axs[i].set_ylabel('Amplitude', fontsize=10)
    axs[i].set_title(
        f'Cutoff Frequency = {fc} Hz', fontsize=11, fontweight='bold')
    axs[i].legend(loc='upper right', fontsize=9)
    axs[i].grid(True, alpha=0.3)
    axs[i].set_ylim([-2, 2])

axs[-1].set_xlabel('Time (seconds)', fontsize=11)
plt.tight_layout()
plt.show()

### REFLECTION QUESTION SET 4: Filtering Demo Reflection (Answer Below)

1. **Cutoff frequency selection:** For the toy signal with a 5 Hz sine wave embedded in noise, which cutoff frequency (2, 5, 10, 50, or 100 Hz) best preserves the low-frequency content while effectively taming the high-frequency noise?

2. **Trade-offs in filtering:** What trade-off do you observe between using a very low cutoff (e.g., 2 Hz) versus a very high cutoff (e.g., 50 Hz)? How does each choice affect:
   - Amplitude preservation of the 5 Hz signal
   - Noise reduction
   - Signal shape and smoothness

3. **The "3 dB rule":** Refer to the table in the figure above. As a multiple of signal frequency, what should your cutoff frequency be to ensure that your signal has approximately zero attenuation? Given a single frequency input signal of 5 Hz, what is the minimum cutoff frequency you would choose based on this rule? Do the plots of the sine wave above support this guideline?

4. **Sampling rate dependency:** Given what you know about postural sway and physiological tremor frequencies, what is the minimum sampling rate you would recommend when collecting force plate data for balance analysis? Justify your answer based on the Nyquist theorem.

### ANSWERS FOR SET 4:

1.

2.

3.

4.

### C. Experiment with Cutoff Frequency

Use this interactive widget to experiment with different cutoff frequencies on the Eyes Open force data. Observe how changing the cutoff frequency affects the filtered signal. Try to find a cutoff that balances noise reduction and signal preservation and use it to answer the reflection questions below.

In [ ]:
# Use the slider to see how using different cutoff frequencies effects the signal
cutoff_freq = widgets.FloatSlider(
    value=100, min=1, max=20, step=0.1, description='Cutoff Frequency (Hz):')

# This is a helper function for filtering a Pandas dataframe


def filter_timeseries_data(data, sampling_freq, custom_cutoff_frequency=None, threshold=99, plot=False):
    """
    Filter time series data using a low-pass Butterworth filter. This function applies a filter to each column of the input data besides the column named 'Time', based on a specified or calculated cutoff frequency.

    Args:
        data (DataFrame): The input time series data to be filtered. Must be a NumPy array or a pandas DataFrame.
        sampling_freq (float): The sampling frequency of the input data.
        custom_cutoff_frequency (float, optional): A user-defined cutoff frequency for the filter. If not provided, it will be determined using spectral analysis.

    Returns:
        DataFrame: The filtered time series data, with the same structure as the input.

    Raises:
        ValueError: If the input data is not a pandas DataFrame.
    Notes:
        The 'Time' column in the data DataFrame is not filtered.
    Examples:
        >>> filtered_data = filter_timeseries_data(data, 1000)
    """
    if not isinstance(data, (pd.DataFrame)):
        raise ValueError(
            "Unsupported data type. Please provide a pandas DataFrame.")

    fs = float(sampling_freq)

    def apply_filter(column, cutoff_frequency):
        b, a = butter(N=4, Wn=cutoff_frequency,
                      btype="low", fs=fs, output="ba")
        return filtfilt(b, a, column)

    if custom_cutoff_frequency is None:
        cutoff_frequencies = {column: float(spectral_analysis(
            data[column], column, sampling_freq, threshold=threshold, plot=plot)) for column in data.columns}
    else:
        cutoff_frequencies = {
            column: custom_cutoff_frequency for column in data.columns}

    filtered_data = data.apply(lambda column: apply_filter(
        column, cutoff_frequencies[column.name]) if column.name != "Time" else column)

    return filtered_data


def filter_and_plot(cutoff_freq, sampling_freq=1000):
    # First filter each dataframe
    filtered_data_EO = filter_timeseries_data(
        df_EO, sampling_freq=sampling_freq, custom_cutoff_frequency=cutoff_freq, plot=False)
    filtered_data_EC = filter_timeseries_data(
        df_EC, sampling_freq=sampling_freq, custom_cutoff_frequency=cutoff_freq, plot=False)

    # Next plot both filtered dataframes
    plot_data(df_EO=filtered_data_EO, df_EC=filtered_data_EC, df_cols=Y_cols, plot_titles=plot_titles,
              y_labels=y_labels, time_range=(0, 5), dpi=100, figsize=(10, 8))


out = widgets.interactive_output(
    filter_and_plot, {'cutoff_freq': cutoff_freq})
print("Note: it may take a minute for the plot to update after moving the slider/prescribing a new cutoff frequency.\nTry a few cutoff frequencies <10 Hz.")
display(cutoff_freq, out)

#### REFLECTION QUESTION SET 5: Interactive Filtering Experiment (Answer Below)

1. Put the slider at 1 Hz. What do you notice about the smoothness and amplitude of the signals compared to when the slider is set close to the maximum value on the slider?

2. Choose a cutoff frequency that you believe maximizes smoothness with minimal signal attenuation, list it, and justify it in one sentence. Do you feel that it's difficult to tell which signal components are noise versus evidence of actual postural sway?

3. Would you use a single cutoff for all channels or channel-specific cutoffs based on these plots? Explain.

#### ANSWERS FOR SET 5:

1.

2.

3.

### D. Understanding Signals in the Frequency Domain

Most biomechanical signals are best understood by examining them in both the **time domain** (how the signal changes over time) and the **frequency domain** (which frequencies contain the signal's energy).

**Time Domain:** Shows amplitude vs. time—what you see in a typical plot.

**Frequency Domain:** Shows power or amplitude vs. frequency. 

To convert from time to frequency domain, we use the **Fast Fourier Transform (FFT)**, which decomposes a signal into its frequency components.


$X[k] = \sum_{n=0}^{N-1} x[n]\,e^{-i\,2\pi kn/N}, \quad k = 0,1,\dots,N-1$


Inverse transform (reconstructing the time signal):

$x[n] = \frac{1}{N}\sum_{k=0}^{N-1} X[k]\,e^{i\,2\pi kn/N}, \quad n = 0,1,\dots,N-1$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

# Use vertical force channel
channel = 'Fz'

# Sampling Rate
fs = sampling_rate

# Limit to first 10 seconds for clarity and convert to millivolts
mask = df_EO['Time'] <= 5
sig_raw = df_EO.loc[mask, channel].to_numpy()
t = df_EO.loc[mask, 'Time'].to_numpy()

# Prepare detrended signal
# We remove the mean (DC offset) from the signal before performing the FFT to avoid a large spike at zero frequency (0 Hz) in the frequency spectrum.
# This DC component does not represent meaningful oscillatory motion—it's just the average value of the signal.
# Removing it allows us to focus on the true dynamic (oscillatory) content of the signal, which is what we care about in biomechanical analysis.
sig_detrended = sig_raw - np.mean(sig_raw)

# Single-sided FFT magnitudes for both versions
freqs = np.fft.rfftfreq(len(sig_detrended), d=1/fs)
fft_det = np.fft.rfft(sig_detrended)
fft_det_mag = (2.0 / len(sig_detrended)) * np.abs(fft_det)


fig, (ax_time, ax_fft_det) = plt.subplots(
    2, 1, figsize=(10, 10), dpi=100)

# Time-domain plot
ax_time.plot(t, sig_raw, color='tab:blue', linewidth=1)
ax_time.set_title(
    f"Time Domain: {channel} (first 5 s, Eyes Open)", fontsize=12, fontweight='bold')
ax_time.set_xlabel('Time (s)')
ax_time.set_ylabel('Force [N]')
ax_time.grid(True, alpha=0.3)

# FFT with mean removed
ax_fft_det.plot(freqs, fft_det_mag, color='tab:red', linewidth=1)
ax_fft_det.set_xlim(0, 50)
ax_fft_det.set_title(
    f"FFT Magnitude (mean-removed): {channel} (Eyes Open)", fontsize=12, fontweight='bold')
ax_fft_det.set_xlabel('Frequency (Hz)')
ax_fft_det.set_ylabel('Amplitude')
ax_fft_det.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### E. Understanding the Power Spectrum

A **power spectrum** tells us:
- Which frequencies contain most of the signal energy
- Where noise dominates
- The optimal cutoff frequency for filtering

We use the **power spectrum** (which shows where your signal's energy is concentrated) to choose an appropriate cutoff frequency. In this notebook we plot **power spectral density (PSD)**: power per Hz, expressed in decibels (dB/Hz).

**What are decibels?** Decibels (dB) represent a logarithmic scale: power in dB = 10 × log₁₀(power in mV²/Hz). Logarithmic scaling compresses wide ranges of values, making small signals visible alongside large ones. For example, if one frequency has 10× the power of another, the dB difference is only 10 dB—much easier to see on a plot than the raw ratio.

**Why use decibels for PSD?** Body sway dominates at very low power (1–5 Hz), while sensor noise spreads across higher frequencies at much smaller power levels. On a linear scale, the low-power noise would be hard to see. On a dB scale, both signal and noise are visible, helping you identify where to set the filter cutoff.

**How PSD is computed here:**
- Remove the mean (DC) to avoid a zero-frequency spike.
- Apply FFT or Welch's method to estimate $|X(f)|^2$ across frequencies.
- Normalize by the window length and sampling rate to report power per Hz $P_{xx}$ ($mV^2/Hz$).
- Convert to dB using 10 × log₁₀(Pxx) for visualization.
- Welch's method averages across overlapping windows to reduce variance.

By examining the PSD in dB, we can visually identify where signal energy drops off and noise begins—this is the ideal cutoff frequency.

Human postural sway typically happens at low frequencies compared to sources of noise.

Human body movements during quiet standing:
- **Postural sway**: 0.5–2 Hz
- **Voluntary corrective movements**: 2–5 Hz  
- **Muscle tremor (physiological)**: 8–12 Hz

Noise sources:
- **Electrical noise (60 Hz line noise)**: 60 Hz and harmonics
- **Sensor/amplifier noise**: Broadband, typically >50 Hz
- **Environmental vibrations**: Variable, often >20 Hz

Let's start by calculating and plotting the PSD for the Eyes Open force data.

In [ ]:
# Frequency-domain view (Power Spectral Density) using Welch on the same real data (now in mV)
sig = sig_raw  # use the mV-converted signal from the earlier FFT cell
f, Pxx = welch(sig, fs=fs, nperseg=4096)

# Convert to decibels
Pxx_dB = 10 * np.log10(Pxx)

fig, ax = plt.subplots(figsize=(10, 4), dpi=100)
ax.plot(f, Pxx_dB, color='tab:green', linewidth=1.5)
ax.set_xlim(0, 100)  # focus on balance-relevant band
ax.set_title(
    f"Frequency Domain: Power Spectrum of {channel} (Eyes Open)", fontsize=12, fontweight='bold')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power Spectral Density (dB/Hz)')
ax.grid(True, which='both', alpha=0.3)
ax.legend([channel])
plt.tight_layout()
plt.show()

### F. Spectral Analysis

**What it is:** Use the power spectral density (PSD) to see where your signal’s energy lives across frequency. Welch’s method (windowed, averaged FFTs) gives a smoother PSD than a single FFT.

**Why it matters:** Balance signals are low-frequency while sensor/environmental noise are higher frequency. A good cutoff keeps the low band and trims the high band.

**Cumulative power approach:** Integrate the PSD from 0 Hz upward to get cumulative power (% of total). Pick the frequency where cumulative power crosses a threshold (commonly 95–99%). That frequency is a data-driven candidate cutoff.

**Practical steps:**
- Compute PSD (Welch) with a window long enough to resolve low frequencies (e.g., 1–4 s windows for balance).

- Compute cumulative power (%) and find where it reaches 95–99%.
- Add a small safety margin above that point (e.g., +1–2 Hz) to avoid shaving real signal, but keep it well below Nyquist (≤0.4–0.5× Nyquist).
- If multiple channels differ, choose the lowest reasonable cutoff across them or apply channel-specific cutoffs.

**Things to remember:** 
- Don't ignore Nyquist frequency (cutoff must be well below fs/2)
- consider using channel-specific filters when some channels are much noisier.

#### Practice Spectral Analysis on Force Plate Data

In [ ]:
# This a helper function for determining cutoff frequencies using cumulative power spectrum thresholding
def spectral_analysis(
    data, column_name, sampling_freq, nperseg=None, window="hann", threshold=99, plot=False
):
    """
    Perform spectral analysis on the given data using the Welch method. This function computes the power spectral density and cumulative power spectrum, optionally plotting the results. Cutoff frequency is selected based on a cumulative power threshold.

    Args:
        data (array-like): The input signal data for which spectral analysis is to be performed.
        sampling_freq (float): The sampling frequency of the input data.
        nperseg (int, optional): Length of each segment for the Welch method. If None, defaults to the length of the data.
        window (str, optional): The window function to use. Defaults to 'hann'.
        threshold (float): The percentage threshold for determining the cutoff frequency in the cumulative power spectrum.
        plot (bool, optional): If True, generates plots for the power spectral density and cumulative power spectrum. Defaults to False.

    Returns:
        float: The frequency at which the cumulative power spectrum exceeds the specified threshold.

    Raises:
        ValueError: If the input data is empty or if the sampling frequency is non-positive.

    Examples:
        >>> freq = spectral_analysis(data, 1000, threshold=95, plot=True)
"""
    if len(data) == 0:
        raise ValueError("Input data is empty")

    if sampling_freq <= 0:
        raise ValueError("Sampling frequency must be positive")

    if nperseg is None:
        nperseg = len(data)

    f, Pxx = welch(data, fs=sampling_freq, nperseg=nperseg, window=window)

    # Calculate cumulative power spectrum
    cumulative_power = np.cumsum(Pxx)

    # Normalize cumulative power to get a percentage
    cumulative_power_percent = cumulative_power / cumulative_power[-1] * 100

    # Find the index where the cumulative power levels off
    cutoff_index = np.argmax(
        cumulative_power_percent > threshold
    )  # Use the specified threshold

    if plot == True and column_name != "Time":
        # Plot the results
        plt.figure(figsize=(10, 8), dpi=100)
        plt.suptitle(f'Spectral Analysis for: {column_name}')
        # Plot Power Spectral Density
        plt.subplot(2, 1, 1)
        plt.semilogy(f, Pxx)
        plt.title('Power Spectral Density')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power/Frequency (dB/Hz)')
        plt.grid(True)

        # Plot Cumulative Power Spectrum
        plt.subplot(2, 1, 2)
        plt.plot(f, cumulative_power_percent)
        plt.axvline(x=f[cutoff_index], color='r', linestyle='--',
                    label=f'Cutoff Frequency: {f[cutoff_index]:.2f} Hz')
        plt.title('Cumulative Power Spectrum')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Cumulative Power (%)')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()
    return f[cutoff_index]


forces_EO = df_EO[['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']]
forces_EC = df_EC[['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']]

# Analyze EO Fz signal at 95% threshold
Fz_cutoff_95 = spectral_analysis(forces_EO['Fz'].values, 'Fz (Vertical Force)', 1000,
                                 threshold=95, plot=True)
print(f"✓ Cutoff frequency (95% power): {Fz_cutoff_95:.1f} Hz")

# Analyze EO Fy signal at 95% threshold
Fy_cutoff_95 = spectral_analysis(forces_EO['Fy'].values, 'Fy (Horizontal Force)', 1000,
                                 threshold=95, plot=True)
print(f"✓ Cutoff frequency (95% power): {Fy_cutoff_95:.1f} Hz")

# Analyze EC Fz signal at 95% threshold
Fz_EC_cutoff_95 = spectral_analysis(forces_EC['Fz'].values, 'Fz (Vertical Force)', 1000,
                                    threshold=95, plot=True)
print(f"✓ Cutoff frequency (95% power): {Fz_EC_cutoff_95:.1f} Hz")

Now let's filter the Eyes Open force data using a Butterworth filter with a cutoff frequency determined from our spectral analysis above. Run the cell below to apply the filter and plot the results.

In [ ]:
fz = df_EO['Fz'].to_frame()

fz_filtered = filter_timeseries_data(
    fz, custom_cutoff_frequency=Fz_cutoff_95, sampling_freq=sampling_freq)


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, dpi=100)

# Plot original signal
ax1.plot(df_EO['Time'], fz, color='tab:blue', linewidth=1, label='Original')
ax1.set_ylabel('Force [N]')
ax1.set_title('Original Fz Signal (Eyes Open)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot filtered signal
ax2.plot(df_EO['Time'], fz_filtered, color='tab:red', linewidth=1,
         label=f'Filtered ({Fz_cutoff_95:.1f} Hz cutoff)')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Force [N]')
ax2.set_title(
    f'Filtered Fz Signal (Cutoff = {Fz_cutoff_95:.1f} Hz)', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
plt.xlim(0, 2)
plt.tight_layout()
plt.show()

#### REFLECTION QUESTION SET 6: Spectral Analysis (Answer Below)

1. At what frequency does EO Fz cumulative power hit 95%? How does that compare to Fy? What might explain any differences?

2. In this analysis, we're assuming most of our signal's power comes from the low frequency components associated with postural sway. If we set our cumulative power threshold very high (e.g. 99%), what assumptions are we making about the amount of noise in our signal?

#### ANSWERS FOR SET 6:

1.

2.

## IV. Summary: Key Concepts Before Lab

### Force Plate Basics
- Force plates measure 6 degrees of freedom: 3 forces (Fx, Fy, Fz) and 3 moments (Mx, My, Mz)
- Raw output is voltages, not forces - a calibration matrix converts voltages to forces
- The center of pressure (CoP) is calculated from force and moment signals
- CoP movement tells us about postural stability and balance control

### Signal Processing
- Real balance signals contain low-frequency components (< 10 Hz)
- Sensor noise is high-frequency (> 50 Hz)
- A low-pass filter removes high-frequency noise while preserving low-frequency motion
- Zero-phase filtering preserves the signal's timing and shape

### Choosing Filter Cutoff Frequencies
- Cutoff frequency must be high enough to preserve real body movement
- Cutoff frequency must be low enough to remove sensor noise
- Spectral analysis (cumulative power) helps us choose an appropriate cutoff
- For balance data, cutoff frequencies are typically 4-15 Hz depending on the signal

### What's Next in Lab
In the actual lab, you will:
1. Perform a tandem stance balance test on the force plate
2. Export raw voltage data from the force plate software
3. Convert voltage signals to forces using the calibration matrix
4. Apply filtering to remove sensor noise
5. Calculate center of pressure and balance metrics
6. Analyze differences between Eyes Open and Eyes Closed conditions

Good luck with the lab! Come prepared with the concepts you've learned here.


# V. Export Notebook to HTML for Submission

To export this notebook as an HTML file for submission, follow these steps:



In [ ]:
# This is a helper function that exports Colab notebook as an HTML file
def export_current_notebook(notebook_path, output_filename="exported_notebook.html"):
    """
    Exports the specified Colab notebook (with all cell outputs) as an HTML file.

    Parameters:
    - notebook_name (str): The name of the notebook file (with extension) to export.
    - output_filename (str): The name of the output HTML file (default is "exported_notebook.html").

    This function reads the notebook file from the current working directory, converts it to an HTML file,
    and saves the output with the given filename. Optionally, it also initiates a download of the HTML file.
    """
    try:

        # Convert to HTML
        with open(notebook_path, "r", encoding="utf-8") as f:
            notebook_content = nbformat.read(f, as_version=4)

        # Convert notebook to HTML
        html_exporter = HTMLExporter()

        # Ensure that input and output cells are included in the exported HTML
        html_exporter.exclude_input = False  # Include code input
        html_exporter.exclude_output = False  # Ensure that outputs are captured

        # Export the notebook to HTML
        html_data, _ = html_exporter.from_notebook_node(notebook_content)

        # Add CSS to wrap lines and prevent clipping
        wrap_css = """
        <style>
            pre, code {
                word-wrap: break-word;
                white-space: pre-wrap;
                word-break: break-word;
            }
        </style>
        """

        # Insert the CSS at the top of the HTML document
        html_data = wrap_css + html_data

        # Save the HTML file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(html_data)

        print(f"Notebook exported successfully as {output_filename}")

        # Optionally download the file
        files.download(output_filename)

    except Exception as e:
        print(f"An error occurred: {e}")


# Make sure this is the correct path to your notebook. If you followed the directions at the beginning, it should already be correct.
notebook_to_export = '/content/drive/MyDrive/ME481-BalanceLab/BalancePreLabAssignment.ipynb'
# This will be the filename of the exported notebook. It will save to your downloads folder locally
output_filename = '/BalancePreLabAssignment.html'

export_current_notebook(notebook_to_export, output_filename)